In [72]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import re

In [6]:
# Load Pretrained Model
path = '../Pretrained_Models/'
w2v_model = Word2Vec.load('../Pretrained_Models/TREC_pretrain.model')

pretrained_weights = w2v_model.wv.vectors
vocab_size, embedding_size = pretrained_weights.shape

print("Vocab Size: ", vocab_size)
print("Embedding Size: ", embedding_size)

Vocab Size:  9447
Embedding Size:  100


In [37]:
path = '../Datasets/Processed/TREC'

training_dev_df = pd.read_csv(f'{path}/train.dev.csv')
training_df = pd.read_csv(f'{path}/train.csv')
test_df = pd.read_csv(f'{path}/test.csv')
print("Training Size: ", training_df.shape[0])
print("Development Size: ", training_dev_df.shape[0])
print("Test Size: ", test_df.shape[0])

Training Size:  3861
Development Size:  500
Test Size:  1091


In [61]:
label_encoder = LabelEncoder()

training_df['label-coarse'] = label_encoder.fit_transform(training_df['label-coarse'])
training_dev_df['label-coarse'] = label_encoder.fit_transform(training_dev_df['label-coarse'])
test_df['label-coarse'] = label_encoder.fit_transform(test_df['label-coarse'])

In [63]:
# Tokenize the text
tokenizer = Tokenizer()
#tokenizer.fit_on_texts(training_dev_df['text'])
tokenizer.fit_on_texts(training_df['text'])

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(training_df['text'])
X_val_sequences = tokenizer.texts_to_sequences(training_dev_df['text'])
X_test_sequences = tokenizer.texts_to_sequences(test_df['text'])

In [64]:
# Padding sequences
sequence_length = 50  # Choose an appropriate sequence length
X_train_padded = pad_sequences(X_train_sequences, maxlen=sequence_length, padding='post')
X_val_padded = pad_sequences(X_val_sequences, maxlen=sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=sequence_length, padding='post')

# Convert labels to one-hot encoded format
y_train = to_categorical(training_df['label-coarse'])
y_val = to_categorical(training_dev_df['label-coarse'])
y_test = to_categorical(test_df['label-coarse'])

In [102]:
num_classes = 5

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=sequence_length))
model.add(LSTM(units=100))  # You can adjust the number of LSTM units
#model.add(LSTM(units=100, activation="relu", kernel_regularizer=regularizers.L1L2(l1=0.001, l2=0.001)))  # You can adjust the number of LSTM units
model.add(Dense(units=256, activation= "relu", kernel_regularizer=regularizers.L1L2(l1=0.0025, l2=0.0025)))
model.add(Dropout(0.01))          
#model.add(Dense(units=256, activation= "relu", kernel_regularizer=regularizers.L1L2(l1=0.0025, l2=0.0025)))
#model.add(Dropout(0.01))
#model.add(Dense(units=256, activation= "relu", kernel_regularizer=regularizers.L1L2(l1=0.0025, l2=0.0025)))
#model.add(Dropout(0.01))
model.add(Dense(units=num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(patience=25)

In [103]:
num_epochs = 1000
batch_size = 100

model.fit(X_train_padded, y_train, epochs=num_epochs, batch_size=batch_size, shuffle=True, validation_data = (X_val_padded, y_val), 
          callbacks = early_stopping, workers = 4)

Epoch 1/1000
39/39 [==============================] - 3s 45ms/step - loss: 4.7580 - accuracy: 0.4465 - val_loss: 3.7041 - val_accuracy: 0.4220
Epoch 2/1000
39/39 [==============================] - 2s 39ms/step - loss: 2.9768 - accuracy: 0.4530 - val_loss: 2.3670 - val_accuracy: 0.4220
Epoch 3/1000
39/39 [==============================] - 2s 39ms/step - loss: 1.9525 - accuracy: 0.4530 - val_loss: 1.6698 - val_accuracy: 0.4220
Epoch 4/1000
39/39 [==============================] - 2s 39ms/step - loss: 1.5204 - accuracy: 0.4530 - val_loss: 1.4726 - val_accuracy: 0.4220
Epoch 5/1000
39/39 [==============================] - 2s 40ms/step - loss: 1.4173 - accuracy: 0.4530 - val_loss: 1.4270 - val_accuracy: 0.4220
Epoch 6/1000
39/39 [==============================] - 2s 40ms/step - loss: 1.3939 - accuracy: 0.4530 - val_loss: 1.4054 - val_accuracy: 0.4220
Epoch 7/1000
39/39 [==============================] - 2s 39ms/step - loss: 1.3814 - accuracy: 0.4530 - val_loss: 1.4072 - val_accuracy: 0.4220

Epoch 58/1000
39/39 [==============================] - 2s 40ms/step - loss: 1.3487 - accuracy: 0.4530 - val_loss: 1.3771 - val_accuracy: 0.4220
Epoch 59/1000
39/39 [==============================] - 2s 40ms/step - loss: 1.3489 - accuracy: 0.4530 - val_loss: 1.3746 - val_accuracy: 0.4220
Epoch 60/1000
39/39 [==============================] - 2s 40ms/step - loss: 1.3484 - accuracy: 0.4530 - val_loss: 1.3632 - val_accuracy: 0.4220
Epoch 61/1000
39/39 [==============================] - 2s 40ms/step - loss: 1.1725 - accuracy: 0.4595 - val_loss: 1.1583 - val_accuracy: 0.5020
Epoch 62/1000
39/39 [==============================] - 2s 40ms/step - loss: 0.9593 - accuracy: 0.6208 - val_loss: 1.0266 - val_accuracy: 0.5700
Epoch 63/1000
39/39 [==============================] - 2s 39ms/step - loss: 0.8277 - accuracy: 0.6496 - val_loss: 0.9844 - val_accuracy: 0.5740
Epoch 64/1000
39/39 [==============================] - 2s 39ms/step - loss: 0.7651 - accuracy: 0.6734 - val_loss: 0.9737 - val_accuracy:

In [104]:
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

35/35 [==============================] - 0s 4ms/step - loss: 0.8841 - accuracy: 0.8048
Test Loss: 0.8840816020965576, Test Accuracy: 0.8047662973403931
